In [1]:
import h5py
import tensorflow as tf
import pandas as pd
from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras.models import Model, load_model

In [2]:
# Load h5 data
with h5py.File('data\\train_df_30.h5', 'r') as f:
    x_train = f['x'][:]
    y_train = f['y'][:]
with h5py.File('data\\validation_df_30.h5', 'r') as f:
    x_validation = f['x'][:]
    y_validation = f['y'][:]
with h5py.File('data\\test_unit_13_30.h5', 'r') as f:
    x_test_13 = f['x'][:]
    y_test_13 = f['y'][:]
with h5py.File('data\\test_unit_14_30.h5', 'r') as f:
    x_test_14 = f['x'][:]
    y_test_14 = f['y'][:]
with h5py.File('data\\test_unit_15_30.h5', 'r') as f:
    x_test_15 = f['x'][:]
    y_test_15 = f['y'][:]

In [3]:
@tf.keras.saving.register_keras_serializable()
def custom_loss(y_true, y_pred):
    penalty_weight = .05
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    penalty = tf.reduce_mean(tf.square(tf.maximum(y_pred - y_true, 0)))
    total_loss = mse + penalty_weight * penalty
    return total_loss


# Load models
hs_model = load_model('hs_nn_model.keras')
rul_model = load_model('rul_nn_model.keras', custom_objects={'custom_loss': custom_loss})

In [4]:
# Create feature extractors
hs_feature_extractor = Model(inputs=hs_model.input, outputs=hs_model.layers[3].output)
rul_feature_extractor = Model(inputs=rul_model.input, outputs=rul_model.layers[3].output)

In [5]:
hs_feature_extractor.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_Layer (InputLayer)        │ (None, 30, 18)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1D_Layer1 (Conv1D)          │ (None, 30, 512)        │        27,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm_Layer1                │ (None, 30, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GlobalAvgPool_Layer             │ (None, 512)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,696 (116.00 KB)

 Trainable params: 28,672 (112.00 KB)

 Non-trainable params: 1,024 (4.00 KB)

In [6]:
rul_feature_extractor.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_Layer (InputLayer)        │ (None, 30, 18)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1D_Layer1 (Conv1D)          │ (None, 30, 512)        │        27,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm_Layer1                │ (None, 30, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ GlobalAvgPool_Layer             │ (None, 512)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,696 (116.00 KB)

 Trainable params: 28,672 (112.00 KB)

 Non-trainable params: 1,024 (4.00 KB)

In [ ]:
# Create feature extracted datasets
hs_x_train_conv = hs_feature_extractor.predict(x_train)
hs_x_validation_conv = hs_feature_extractor.predict(x_validation)
hs_x_test_13_conv = hs_feature_extractor.predict(x_test_13)
hs_x_test_14_conv = hs_feature_extractor.predict(x_test_14)
hs_x_test_15_conv = hs_feature_extractor.predict(x_test_15)

rul_x_train_conv = rul_feature_extractor.predict(x_train)
rul_x_validation_conv = rul_feature_extractor.predict(x_validation)
rul_x_test_13_conv = rul_feature_extractor.predict(x_test_13)
rul_x_test_14_conv = rul_feature_extractor.predict(x_test_14)
rul_x_test_15_conv = rul_feature_extractor.predict(x_test_15)

In [ ]:
# Convert to pd dataframes and save as h5
hs_x_train_conv = pd.DataFrame(hs_x_train_conv)
hs_x_validation_conv = pd.DataFrame(hs_x_validation_conv)
hs_x_test_13_conv = pd.DataFrame(hs_x_test_13_conv)
hs_x_test_14_conv = pd.DataFrame(hs_x_test_14_conv)
hs_x_test_15_conv = pd.DataFrame(hs_x_test_15_conv)

rul_x_train_conv = pd.DataFrame(rul_x_train_conv)
rul_x_validation_conv = pd.DataFrame(rul_x_validation_conv)
rul_x_test_13_conv = pd.DataFrame(rul_x_test_13_conv)
rul_x_test_14_conv = pd.DataFrame(rul_x_test_14_conv)
rul_x_test_15_conv = pd.DataFrame(rul_x_test_15_conv)


In [ ]:
# Save feature extracted datasets
with h5py.File('data\\hs_x_train_conv.h5', 'w') as f:
    f.create_dataset('train', data=hs_x_train_conv)
    f.create_dataset('validation', data=hs_x_validation_conv)
with h5py.File('data\\hs_x_test_13_conv.h5', 'w') as f:
    f.create_dataset('test13', data=hs_x_test_13_conv)
with h5py.File('data\\hs_x_test_14_conv.h5', 'w') as f:
    f.create_dataset('test14', data=hs_x_test_14_conv)
with h5py.File('data\\hs_x_test_15_conv.h5', 'w') as f:
    f.create_dataset('test15', data=hs_x_test_15_conv)

with h5py.File('data\\rul_x_train_conv.h5', 'w') as f:
    f.create_dataset('train', data=rul_x_train_conv)
    f.create_dataset('validation', data=rul_x_validation_conv)
with h5py.File('data\\rul_x_test_13_conv.h5', 'w') as f:
    f.create_dataset('test13', data=rul_x_test_13_conv)
with h5py.File('data\\rul_x_test_14_conv.h5', 'w') as f:
    f.create_dataset('test14', data=rul_x_test_14_conv)
with h5py.File('data\\rul_x_test_15_conv.h5', 'w') as f:
    f.create_dataset('test15', data=rul_x_test_15_conv)

In [7]:
# Save feature extractors
hs_feature_extractor.save('hs_feature_extractor.keras')
rul_feature_extractor.save('rul_feature_extractor.keras')